In [1]:
from astropy import units as u
from astropy.coordinates import SkyCoord
import astropy.coordinates as coord
import numpy as np
import matplotlib.pyplot as plt
from astropy_healpix import HEALPix
import csv
from scipy.special import gamma, factorial
import ipywidgets as widgets
from IPython.display import display
from IPython.display import clear_output

In [2]:
def lim(r,rlo,rhi):
    return 1*np.bitwise_and(r>=rlo,r<=rhi) 

def prior4(r,rlen,alpha,beta):
    r = np.where(r > 0,r,0)
    return 1/gamma((beta+1)/alpha)*alpha/(rlen**(beta+1))*r**beta*np.exp(-((r/rlen)**alpha))

def prior3(r,rlen):
    return lim(r=r,rlo=0,rhi=np.inf)*np.exp(-r/rlen)*r**2

def onclick(event): 
    #Removing previous figure
    
    ax2.cla()  
    ax.cla() 
    del fig.texts[:]
    
    #Add colormap,ticklabels,grid
    
    im = ax.pcolormesh(X,Y,Z,cmap = 'plasma')
    ax.set_xticklabels(['150°','120°','90°','60°','30°','0°','330°','300°','270°','240°','210°'],color = 'black')
    ax.grid(linewidth=0.3)
    
    #Get parameters
    
    l = -event.xdata * 180/np.pi
    b = event.ydata * 180/np.pi 
                
    if l <= 0:
        l = l +360   
    
    print('glon [deg]:',f'{l:.3f}' ,', glat [deg]:',f'{b:.3f}') 
    fig.text(0.05, 0.27,f'glon [deg]: {l:.3f}')
    fig.text(0.05, 0.24,f'glat [deg]: {b:.3f}')
             
    gc = SkyCoord(l=l*u.degree, b=b*u.degree, frame='galactic')
    
    ra = gc.fk5.ra.degree
    
    dec = gc.fk5.dec.degree
    
    print('ra [deg]:',f'{ra:.3f}',', dec [deg]:',f'{dec:.3f}')
    fig.text(0.05, 0.21,f'ra [deg]: {ra:.3f}')
    fig.text(0.05, 0.18,f'dec [deg]: {dec:.3f}')
    
    hp = HEALPix(nside=2**5, order='nested') #level 5
    HP = hp.lonlat_to_healpix(ra*u.deg,dec*u.deg)
    print('HEALPix level 5:',HP)
    fig.text(0.05, 0.15,f'HEALPix level 5: {HP}')
    
    with open('prior_summary.csv', newline='') as f:
        reader = csv.reader(f)
        rows = []
        for row in reader:
            rows.append(row)    
        rlen_EDSD = float(rows[HP+1][10]) #EDSDrlen in csv-file  
        rlen_GGD = float(rows[HP+1][5]) # GGDrlen in csv-file
        alpha = float(rows[HP+1][6])
        beta = float(rows[HP+1][7])
        glon = float(rows[HP+1][1])
        glat = float(rows[HP+1][2])
        mode = float(rows[HP+1][9])
        
        print('glon [deg] for HEALPix:',glon,', glat [deg] for HEALPix',glat)
        
        fig.text(0.05, 0.12,f'glon [deg] for HEALPix: {glon:.3f}')
        fig.text(0.05, 0.09,f'glat [deg] for HEALPix: {glat:.3f}')
        
    
        gc0 = SkyCoord(l=glon*u.degree, b=glat*u.degree, frame='galactic')
        print('ra [deg] for HEALPix:',f'{gc0.icrs.ra.degree:.3f}',', dec [deg] for HEALPix:',f'{gc0.icrs.dec.degree:.3f}')
        print('-----------------------------------------------------------------------------------')   
        
        fig.text(0.05, 0.06,f'ra [deg] for HEALPix: {gc0.icrs.ra.degree:.3f}')
        fig.text(0.05, 0.03,f'dec [deg] for HEALPix: {gc0.icrs.dec.degree:.3f}') 
        
        if l <= 180:
            ax.scatter(np.deg2rad(-l),np.deg2rad(b))
        if l > 180:
            ax.scatter(np.deg2rad(-l+360),np.deg2rad(b))
        
        
        ax2.grid()
        
        if Check == True:
            rplotlo = Ran[0]*1e3 
            rplothi = Ran[1]*1e3
        else:
            rplotlo = 0 
            rplothi = 10*mode
        Nplot = 1e3
        s = np.arange(1/(2*Nplot),1/Nplot*(Nplot+1),1/Nplot)
        rplot = s*(rplothi-rplotlo) + rplotlo
        
        if Model == 'GGD':
            dprior4 = prior4(r=rplot,rlen=rlen_GGD,alpha=alpha,beta=beta)
            Z4 = sum((1/Nplot*(rplothi-rplotlo) + rplotlo)*dprior4)
            ax2.plot(1e-3*rplot,dprior4/Z4,label = 'GGD Prior')
            
        if Model == 'EDSD':
            dprior3 = prior3(r=rplot,rlen=rlen_EDSD)
            Z3 = sum((1/Nplot*(rplothi-rplotlo) + rplotlo)*dprior3)
            ax2.plot(1e-3*rplot,dprior3/Z3,label = 'EDSD Prior')
            
        if Model == 'GGD and EDSD':
            
            dprior4 = prior4(r=rplot,rlen=rlen_GGD,alpha=alpha,beta=beta)
            Z4 = sum((1/Nplot*(rplothi-rplotlo) + rplotlo)*dprior4)
            ax2.plot(1e-3*rplot,dprior4/Z4,label = 'GGD Prior')
            
            dprior3 = prior3(r=rplot,rlen=rlen_EDSD)
            Z3 = sum((1/Nplot*(rplothi-rplotlo) + rplotlo)*dprior3)
            ax2.plot(1e-3*rplot,dprior3/Z3,label = 'EDSD Prior')    
            
        ax2.set_title(f'glon:{l:.3f} deg ,glat:{b:.3f} deg')
        ax2.set_xlim([1e-3*rplotlo,1e-3*rplothi])
        ax2.set_xlabel('distance [kpc]')
        plt.legend()
        plt.show()
        

In [3]:
def f(custom,model,glon,glat,inp,HEALpix,check,ran):
    
    if custom == 'interactive input':
        global Check
        Check = check
        global Ran
        Ran = ran
        global Model 
        Model = model
        
        %matplotlib tk
        
        #Setting up figure
        global fig
        fig = plt.figure()
        global ax
        ax = fig.add_subplot(1,2,1,projection="hammer")
        ax.set_xlabel('glat[deg]')
        ax.set_ylabel('glon[deg]')
        global ax2
        ax2 = fig.add_subplot(1,2,2)
        
        # Adding HEALPix colorsceme ------------------------------------------------------------------------------------------
        
        global X,Y,Z
        
        x = np.linspace(-np.pi,np.pi,600)
        y = np.linspace(-np.pi/2,np.pi/2,600)
        
        X,Y = np.meshgrid(x,y)
        
        #l = X*180/np.pi -180
        #b = Y*180/np.pi
        
        l = -x*180/np.pi
        b = y*180/np.pi 
                
        for i in range(len(l)):
            if l[i]<= 0:
                l[i] = l[i]+360
                        
                
        l,b = np.meshgrid(l,b)
        
        
        gc = SkyCoord(l=l*u.degree, b=b*u.degree, frame='galactic')
            
        ra = gc.icrs.ra.degree
            
        dec = gc.icrs.dec.degree
            
        hp = HEALPix(nside=2**5, order='nested') #level 5
        
        HP = hp.lonlat_to_healpix(ra*u.deg,dec*u.deg)
        
        Z = HP
        
        im = ax.pcolormesh(X,Y,Z,cmap = 'plasma')
        plt.colorbar(im,shrink=0.5,label='HEALPix Level 5')
        #----------------------------------------------------------------------------------------------------------------------
        
        ax.set_xticklabels(['150°','120°','90°','60°','30°','0°','330°','300°','270°','240°','210°'],color = 'black')
        ax.grid(linewidth=0.3)
        
        cid = fig.canvas.mpl_connect('button_press_event', onclick)
        
        plt.waitforbuttonpress() #plt.pause(5)
        #fig.canvas.mpl_disconnect(cid)
        fig.tight_layout()
        
    if custom == 'manual input':
        
        
        if inp == 'glon/glat':
            
            if glon>=0 and glon <=360 and glat<=90 and glat >=-90:
               
                l = glon
                b = glat

                print('glon [deg]:',f'{l:.3f}' ,', glat [deg]:',f'{b:.3f}') 
                
                gc = SkyCoord(l=l*u.degree, b=b*u.degree, frame='galactic')
            
                ra = gc.icrs.ra.degree
                dec = gc.icrs.dec.degree
                print('ra [deg]:',f'{ra:.3f}',', dec [deg]:',f'{dec:.3f}')
                hp = HEALPix(nside=2**5, order='nested') #level 5
                HP = hp.lonlat_to_healpix(ra*u.deg,dec*u.deg)
                print('HEALPix level 5:',HP)
            
                with open('prior_summary.csv', newline='') as f:
                    reader = csv.reader(f)
                    rows = []
                    for row in reader:
                        rows.append(row)
                    rlen_EDSD = float(rows[HP+1][10]) #EDSDrlen in csv-file  
                    rlen_GGD = float(rows[HP+1][5]) # GGDrlen in csv-file
                    alpha = float(rows[HP+1][6])
                    beta = float(rows[HP+1][7])
                    glonhp = float(rows[HP+1][1])
                    glathp = float(rows[HP+1][2])
                    mode = float(rows[HP+1][9])
                    
                print('HEALPix data from csv-file:')
                print('glon [deg] for HEALPix:',f'{glonhp:.3f}',', glat [deg] for HEALPix',f'{glathp:.3f}')
                gc0 = SkyCoord(l=glonhp*u.degree, b=glathp*u.degree, frame='galactic')
                print('ra [deg] for HEALPix:',f'{gc0.fk5.ra.degree:.3f}',', dec [deg] for HEALPix:',f'{gc0.fk5.dec.degree:.3f}')
                
                if model == 'GGD':
                    print('GGD Prior Mode:',f'{mode:.3f}')
                    
                print('-----------------------------------------------------------------------------------')   
                
                # Plot Prior
                
                %matplotlib inline
                #plt.style.use('ggplot')    
                plt.rcParams["figure.figsize"] = [7.50, 3.50]
                plt.rcParams["figure.autolayout"] = True   
                
                fig = plt.figure()
                
                ax = fig.add_subplot(projection="hammer")
                ax.set_xlabel('glat[deg]')
                ax.set_ylabel('glon[deg]')
                
                
                # Adding HEALpix colorsceme ---------------------------------------------------------------------------------------
                
                x = np.linspace(-np.pi,np.pi,500)
                y = np.linspace(-np.pi/2,np.pi/2,500)
                
                l1 = -x*180/np.pi
                b1 = y*180/np.pi 
                
                for i in range(len(l1)):
                    if l1[i]<= 0:
                        l1[i] = l1[i]+360
                        
                X,Y = np.meshgrid(x,y)      
                l1,b1 = np.meshgrid(l1,b1)
                
                gc1 = SkyCoord(l=l1*u.degree, b=b1*u.degree, frame='galactic')
                    
                ra1 = gc1.fk5.ra.degree
                    
                dec1 = gc1.fk5.dec.degree
                    
                hp = HEALPix(nside=2**5, order='nested') #level 5
                
                HP1 = hp.lonlat_to_healpix(ra1*u.deg,dec1*u.deg)
        
                Z = HP1
        
                im = ax.pcolormesh(X,Y,Z,cmap = 'plasma')
                plt.colorbar(im,shrink=0.5,label='HEALPix Level 5')
                
                #-----------------------------------------------------------------------------------------------------------------
                
                ax.set_xticklabels(['150°','120°','90°','60°','30°','0°','330°','300°','270°','240°','210°'])
                ax.grid(linewidth=0.3)
                
                if l <= 180:
                    
                    ax.scatter(np.deg2rad(-l),np.deg2rad(b))
                    
                if l > 180:
                    
                    ax.scatter(np.deg2rad(-l+360),np.deg2rad(b))
                
                
                fig2,ax2 = plt.subplots()
                ax2.grid()
                if check == True:
                    rplotlo = ran[0]*1e3
                    rplothi = ran[1]*1e3
                else:
                    rplotlo = 0 
                    rplothi = 10*mode
                    
                Nplot = 1e3
                s = np.arange(1/(2*Nplot),1/Nplot*(Nplot+1),1/Nplot)
                rplot = s*(rplothi-rplotlo) + rplotlo
                
                if model == 'GGD':
                    dprior4 = prior4(r=rplot,rlen=rlen_GGD,alpha=alpha,beta=beta)
                    Z4 = sum((1/Nplot*(rplothi-rplotlo) + rplotlo)*dprior4)
                    ax2.plot(1e-3*rplot,dprior4/Z4,label = 'GGD Prior')
                    
                if model == 'EDSD':
                    dprior3 = prior3(r=rplot,rlen=rlen_EDSD)
                    Z3 = sum((1/Nplot*(rplothi-rplotlo) + rplotlo)*dprior3)
                    ax2.plot(1e-3*rplot,dprior3/Z3,label = 'EDSD Prior')
                    
                if model == 'GGD and EDSD':
                    
                    dprior4 = prior4(r=rplot,rlen=rlen_GGD,alpha=alpha,beta=beta)
                    Z4 = sum((1/Nplot*(rplothi-rplotlo) + rplotlo)*dprior4)
                    ax2.plot(1e-3*rplot,dprior4/Z4,label = 'GGD Prior')
                    
                    dprior3 = prior3(r=rplot,rlen=rlen_EDSD)
                    Z3 = sum((1/Nplot*(rplothi-rplotlo) + rplotlo)*dprior3)
                    ax2.plot(1e-3*rplot,dprior3/Z3,label = 'EDSD Prior')
                    
                ax2.set_title(f'glon:{l:.3f} deg ,glat:{b:.3f} deg')
                ax2.set_xlim([1e-3*rplotlo,1e-3*rplothi])
                ax2.set_xlabel('distance [kpc]')
                plt.legend()
                plt.show()
            
            else:
                print('Error: Input of glon/glat out of range')
                
        if inp == 'HEALpixel': 
            
            if HEALpix >= 0 and HEALpix <= 12287:
                
                HP = HEALpix
                print('HEALPix level 5:',HP)
                
                with open('prior_summary.csv', newline='') as f:
                    reader = csv.reader(f)
                    rows = []
                    for row in reader:
                        rows.append(row)
                    rlen_EDSD = float(rows[HP+1][10]) #EDSDrlen in csv-file  
                    rlen_GGD = float(rows[HP+1][5]) # GGDrlen in csv-file
                    alpha = float(rows[HP+1][6])
                    beta = float(rows[HP+1][7])
                    glonhp = float(rows[HP+1][1])
                    glathp = float(rows[HP+1][2])
                    mode = float(rows[HP+1][9])
                    
                print('HEALPix data from csv-file:')
                print('glon [deg] for HEALPix:',f'{glonhp:.3f}',', glat [deg] for HEALPix',f'{glathp:.3f}')
                gc0 = SkyCoord(l=glonhp*u.degree, b=glathp*u.degree, frame='galactic')
                print('ra [deg] for HEALPix:',f'{gc0.fk5.ra.degree:.3f}',', dec [deg] for HEALPix:',f'{gc0.fk5.dec.degree:.3f}')
                
                if model == 'GGD':
                    print('GGD Prior Mode:',f'{mode:.3f}')
                    
                print('-----------------------------------------------------------------------------------')   
                
                # Plot Prior
                
                %matplotlib inline
                #plt.style.use('ggplot')    
                plt.rcParams["figure.figsize"] = [7.50, 3.50]
                plt.rcParams["figure.autolayout"] = True   
                
                fig = plt.figure()
                
                ax = fig.add_subplot(projection="hammer")
                ax.set_xlabel('glat[deg]')
                ax.set_ylabel('glon[deg]')
                
                
                # Adding HEALpix colorsceme ---------------------------------------------------------------------------------------
                
                x = np.linspace(-np.pi,np.pi,500)
                y = np.linspace(-np.pi/2,np.pi/2,500)
        
                l1 = -x*180/np.pi
                b1 = y*180/np.pi 
                
                for i in range(len(l1)):
                    if l1[i]<= 0:
                        l1[i] = l1[i]+360
                        
                X,Y = np.meshgrid(x,y)        
                l1,b1 = np.meshgrid(l1,b1)
                
                gc1 = SkyCoord(l=l1*u.degree, b=b1*u.degree, frame='galactic')
                    
                ra1 = gc1.fk5.ra.degree
                    
                dec1 = gc1.fk5.dec.degree
                    
                hp = HEALPix(nside=2**5, order='nested') #level 5
                
                HP1 = hp.lonlat_to_healpix(ra1*u.deg,dec1*u.deg)
        
                Z = HP1
        
                im = ax.pcolormesh(X,Y,Z,cmap = 'plasma')
                plt.colorbar(im,shrink=0.5,label='HEALPix Level 5')
                
                #-----------------------------------------------------------------------------------------------------------------
                
                ax.set_xticklabels(['150°','120°','90°','60°','30°','0°','330°','300°','270°','240°','210°'])
                
                ax.grid(linewidth=0.3)
                
                if glonhp <= 180:
                    ax.scatter(np.deg2rad(-glonhp),np.deg2rad(glathp))
                if glonhp > 180:
                    ax.scatter(np.deg2rad(-glonhp+360),np.deg2rad(glathp))
                
                
                fig2,ax2 = plt.subplots()
                ax2.grid()
                
                if check == True:
                    rplotlo = ran[0]*1e3
                    rplothi = ran[1]*1e3
                else:
                    rplotlo = 0 
                    rplothi = 10*mode
                Nplot = 1e3
                s = np.arange(1/(2*Nplot),1/Nplot*(Nplot+1),1/Nplot)
                rplot = s*(rplothi-rplotlo) + rplotlo
                
                if model == 'GGD':
                    
                    dprior4 = prior4(r=rplot,rlen=rlen_GGD,alpha=alpha,beta=beta)
                    Z4 = sum((1/Nplot*(rplothi-rplotlo) + rplotlo)*dprior4)
                    ax2.plot(1e-3*rplot,dprior4/Z4,label = 'GGD Prior')
                    
                if model == 'EDSD':
                    dprior3 = prior3(r=rplot,rlen=rlen_EDSD)
                    Z3 = sum((1/Nplot*(rplothi-rplotlo) + rplotlo)*dprior3)
                    ax2.plot(1e-3*rplot,dprior3/Z3,label = 'EDSD Prior')
                    
                if model == 'GGD and EDSD':
                    
                    dprior4 = prior4(r=rplot,rlen=rlen_GGD,alpha=alpha,beta=beta)
                    Z4 = sum((1/Nplot*(rplothi-rplotlo) + rplotlo)*dprior4)
                    ax2.plot(1e-3*rplot,dprior4/Z4,label = 'GGD Prior')
                    
                    dprior3 = prior3(r=rplot,rlen=rlen_EDSD)
                    Z3 = sum((1/Nplot*(rplothi-rplotlo) + rplotlo)*dprior3)
                    ax2.plot(1e-3*rplot,dprior3/Z3,label = 'EDSD Prior')
                    
                    
                    
                ax2.set_title(f'HEALpixel {HP}')
                ax2.set_xlim([1e-3*rplotlo,1e-3*rplothi])
                ax2.set_xlabel('distance [kpc]')
                plt.legend()
                plt.show()
                
                
            else:
                print('Error: Input of HEALpixel out of range')

In [9]:
model = widgets.RadioButtons(
    options=['GGD','EDSD','GGD and EDSD'],
    description='Model:',
    disabled=False
)

check = widgets.Checkbox(
    value=False,
    description='set plotting range manually:',
    disabled=False
)

ran = widgets.FloatRangeSlider(
    value=[0, 10],
    min=0,
    max=50,
    step=0.1,
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f'
)


custom=widgets.Select(
    options=['interactive input', 'manual input'],
    value='manual input',
    # rows=10,
    description='Mode:',
    disabled=False
)


inp = widgets.Dropdown(
    options=['glon/glat', 'HEALpixel'],
    value='glon/glat',
    description='Input:',
    disabled=False,
)





glon0 = widgets.FloatSlider(
    value=60,
    min=0,
    max=360,
    step=1,
    description='glon [deg]:',
    disabled=False,
    continuous_update=True,
    orientation='horizontal',
    readout=False,
    readout_format='.2f',
    style = {'description_width': 'initial'}
    
)
glon = widgets.FloatText()

widgets.jslink((glon0, 'value'), (glon, 'value'))

glat0 = widgets.FloatSlider(
    value=60,
    min=-90,
    max=90,
    step=1,
    description='glat [deg]:',
    disabled=False,
    continuous_update=True,
    orientation='horizontal',
    readout=False,
    readout_format='.2f',
    style = {'description_width': 'initial'}
    
)
glat = widgets.FloatText()

widgets.jslink((glat0, 'value'), (glat, 'value'))

HEALpix0 = widgets.IntSlider(
    value=50,
    min=0,
    max=12287,
    step=1,
    description='HEALpixel:',
    disabled=False,
    continuous_update=True,
    orientation='horizontal',
    readout=False,
    readout_format='.2f',
    style = {'description_width': 'initial'}
    
)
HEALpix = widgets.IntText()

widgets.jslink((HEALpix0, 'value'), (HEALpix, 'value'))

display(model)
#display(check)
#display(ran)

display(widgets.HBox([check,ran]))
display(custom)
display(inp)

display(widgets.HBox([glon0,glon]))
display(widgets.HBox([glat0,glat]))
display(widgets.HBox([HEALpix0,HEALpix]))

# adding start button --------------------------------------------------------------------------------------------------------

out = widgets.Output()

# submit button

submit_button = widgets.Button(description='start')
display(submit_button)

# submit function

def submit(button):
    out.clear_output()
    with out:
        f(custom = custom.value , model = model.value, glon = glon.value ,glat = glat.value , inp = inp.value, HEALpix = HEALpix.value, check = check.value, ran = ran.value)
        
        
# tie submit button to a function

submit_button.on_click(submit)
display(out)

# for continuous updates rather than start button------------------------------------------------------------------------------

#out = widgets.interactive_output(f,{'custom':custom,'model':model,'glon':glon,'glat':glat,'inp':inp,'HEALpix':HEALpix,'check':check,'ran':ran})
#display(out)


RadioButtons(description='Model:', options=('GGD', 'EDSD', 'GGD and EDSD'), value='GGD')

Select(description='Mode:', index=1, options=('interactive input', 'manual input'), value='manual input')

Dropdown(description='Input:', options=('glon/glat', 'HEALpixel'), value='glon/glat')

Button(description='start', style=ButtonStyle())

Output()